In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [3]:
# web_page_data.csv를 사용해서 A, B 두 페이지의 접속시간 차이를 검정
web_page_df = pd.read_csv('data-files/web_page_data.csv')

In [6]:
# 두 페이지의 접속 시간 차이 확인
web_page_df.groupby('Page')['Time'].mean()

Page
Page A    1.263333
Page B    1.620000
Name: Time, dtype: float64

In [7]:
# pvalue를 근거로 차이 없는 모집단으로부터 위에서 보인 차이가 나타나는 표본이 뽑힐 가능성이 27%
# -> 귀무가설 기각할 수 없음 
# -> 두 페이지의 접속시간에는 차이가 없음
stats.ttest_ind(web_page_df[web_page_df['Page'] == 'Page A']['Time'],
                web_page_df[web_page_df['Page'] == 'Page B']['Time'])

TtestResult(statistic=np.float64(-1.1237042154424823), pvalue=np.float64(0.26901024363926024), df=np.float64(34.0))

In [11]:
tips_df = sns.load_dataset('tips')
tips_df.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [ ]:
# 성별에 따라 tip에 차이가 있는지 분석
# 흡연여부에 따라 tip에 차이가 있는지 분석
# 식사시간에 따라 tip에 차이가 있는지 분석
# 요일에 따라 tip에 차이가 있는지 분석

In [15]:
print( tips_df['sex'].unique() )
print( tips_df['smoker'].unique() )
print( tips_df['time'].unique() )
print( tips_df['day'].unique() )

['Female', 'Male']
Categories (2, object): ['Male', 'Female']
['No', 'Yes']
Categories (2, object): ['Yes', 'No']
['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']
['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']


In [17]:
# 식사시간에 따라 tip에 차이가 있는지 분석
tips_df.groupby('time')['tip'].mean()

C:\Users\human\AppData\Local\Temp\ipykernel_17916\3736541840.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips_df.groupby('time')['tip'].mean()


time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [18]:
stats.ttest_ind(tips_df[tips_df['time'] == 'Lunch']['tip'],
                tips_df[tips_df['time'] == 'Dinner']['tip'])

TtestResult(statistic=np.float64(-1.9062569301202392), pvalue=np.float64(0.05780153475171558), df=np.float64(242.0))

In [19]:
# 요일에 따라 tip에 차이가 있는지 분석
tips_df.groupby('day')['tip'].mean()

C:\Users\human\AppData\Local\Temp\ipykernel_17916\2102971544.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips_df.groupby('day')['tip'].mean()


day
Thur    2.771452
Fri     2.734737
Sat     2.993103
Sun     3.255132
Name: tip, dtype: float64

In [ ]:
weekend_mask = (tips_df['day'] == 'Sat') | (tips_df['day'] == 'Sun')
tips_df[weekend_mask]['tip'].mean(), \
tips_df[~weekend_mask]['tip'].mean(), \
tips_df[weekend_mask]['tip'].mean() - tips_df[~weekend_mask]['tip'].mean()

(np.float64(3.115276073619632),
 np.float64(2.7628395061728392),
 np.float64(0.35243656744679264))

In [ ]:
# pvalue : 0.06 으로 귀무가설 기각할 수 없음 -> 주말과 주중은 tip의 양에 차이가 있다고 주장할 수 없음
stats.ttest_ind(tips_df[weekend_mask]['tip'],
                tips_df[~weekend_mask]['tip'])

TtestResult(statistic=np.float64(1.8834950801996837), pvalue=np.float64(0.06083244523772748), df=np.float64(242.0))

In [ ]:
# pvalue : 0.03 으로 귀무가설 기각할 수 있음 -> 주말에 tip을 더 많이 준다고 주장할 수 있다
stats.ttest_ind(tips_df[weekend_mask]['tip'],
                tips_df[~weekend_mask]['tip'],
                alternative='greater')

TtestResult(statistic=np.float64(1.8834950801996837), pvalue=np.float64(0.03041622261886374), df=np.float64(242.0))

In [34]:
tips_df.shape

(244, 7)

In [28]:
# !pip install statsmodels

In [29]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [30]:
# 요일에 따라 tip에 차이가 있는지 분석
tips_df.groupby('day')['tip'].mean()

C:\Users\human\AppData\Local\Temp\ipykernel_17916\2102971544.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips_df.groupby('day')['tip'].mean()


day
Thur    2.771452
Fri     2.734737
Sat     2.993103
Sun     3.255132
Name: tip, dtype: float64

In [31]:
model = smf.ols("tip ~ day", data=tips_df).fit() # tip = f(day)
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
day,3.0,9.525873,3.175291,1.672355,0.173589
Residual,240.0,455.686604,1.898694,NaN,NaN


In [32]:
model = smf.ols("tip ~ time", data=tips_df).fit() # tip = f(time)
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
time,1.0,6.882181,6.882181,3.633815,0.057802
Residual,242.0,458.330296,1.893927,NaN,NaN
